In [1]:
# Import general python packages used by scientists
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os

# Import packages  Image Access
import lsst.daf.butler as dafButler
import lsst.geom as geom
from lsst.geom import PointD
from lsst.geom import Point2D
import lsst.afw.display as afwDisplay
import lsst.daf.base as dafBase
from lsst.daf.butler import Butler
import lsst.afw.image as afwImage
import lsst.afw.table as afwTable
from lsst.afw.geom.ellipses import Quadrupole, SeparableDistortionTraceRadius
from lsst.afw import cameraGeom

# Import packages for  Catalog Access
import pandas as pd
pd.set_option('display.max_rows', 1000)
from lsst.rsp import get_tap_service, retrieve_query

#Import custom packages
from ellipticity_mapping import calculate_ellipticity_on_xy
from ellipticity_mapping import remove_figure
from ellipticity_mapping import plot_ellipticitymap
from collection_dictionary_shared import collection_dictionary
from collection_dictionary_comcamjosh import collection_dictionary_comcamjosh
from collection_dictionary_shared_passband import collection_dictionary_passband
from collection_dictionary_comcamjosh_passband import collection_dictionary_comcamjosh_passband
from rotation_conversion import rsp_to_rtp

from datetime import datetime, timedelta
from astroplan import Observer
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from pytz import all_timezones #To visualize all the possible timezones
from pytz import timezone
import datetime
import time

import lsst.cbp as cbp

In [2]:
timestamp = time.time()  # Example timestamp
datetime_object = datetime.datetime.fromtimestamp(timestamp)
formatted_datetime = datetime_object.strftime("%Y%m%dT%H%M%S")

In [3]:
def pixel_to_camera(x, y, det):
    """
    Parameters
    ----------
    x, y : array
        Pixel coordinates.
    det : lsst.afw.cameraGeom.Detector
        Detector of interest.
    Returns
    -------
    cam_x, cam_y : array
        Focal plane position in millimeters in DVCS
        See https://lse-349.lsst.io/
    """
    tx = det.getTransform(cameraGeom.PIXELS, cameraGeom.FOCAL_PLANE)
    cam_x, cam_y = tx.getMapping().applyForward(np.vstack((x, y)))
    return cam_x.ravel(), cam_y.ravel()

def pixel_to_camera_angle(x, y, det):
    """
    Parameters
    ----------
    x, y : array
        Pixel coordinates.lsst afw.detectordetectordetectordetector
    det : lsst.afw.cameraGeom.Detector
        Detector of interest.
    Returns
    -------
    cam_x, cam_y : array
        Focal plane position in degrees in DVCS
        See https://lse-349.lsst.io/
    """
    tx = det.getTransform(cameraGeom.PIXELS, cameraGeom.FIELD_ANGLE)
    cam_x, cam_y = tx.getMapping().applyForward(np.vstack((x, y)))
    return np.degrees(cam_x.ravel()), np.degrees(cam_y.ravel())

In [4]:
collection_dictionary_id = 0
# 0 = our Imsim geneerated images
# 1 = ComCam Josh simulations in /sdf/data/rubin/repo/aos_imsim/raw/test_OR4/output/
print("potrei leggere queste cose da yaml???")

if collection_dictionary_id==0:
    visit_ids = [119]
    detectors = list(np.arange(189))
    detectors.pop(27)
    detectors.pop(160)
    # detectors = list(80 + np.arange(100))
    collection_dict = collection_dictionary()
    collection_dict_band = collection_dictionary_passband()
    seqnum_base = 5023071800000
    string_dataset = 'ours'
    figure_size_degrees = 2.

elif collection_dictionary_id==1:
    seqnum_base = 7024062500000
    visit_ids = [99]
    # visit_ids = [7024062500015]
    detectors = list(np.arange(9))
    collection_dict = collection_dictionary_comcamjosh()
    collection_dict_band = collection_dictionary_comcamjosh_passband()
    string_dataset = 'ComCamJosh'
    figure_size_degrees = .5

format_figures = 'png'

potrei leggere queste cose da yaml???


In [5]:
folder = '/sdf/data/rubin/shared/image_quality/imsim/'
#Define the butler data configuration and collection (una tantum )
config = folder+'repo'
folderout = folder+'ellipticitymap/'
subfolderout_fig = 'figures/'
subfolderout_tab = 'tables/'
string_grid_or_star = ['grid', 'stars', 'detcenter']

In [6]:
# Saves to a file the information (seqnum, detector and collection) 
# associated to each run of this notebook and each output file
filedict = open(folderout+'mean_ellipticity_dictionary', 'a')  # append mode

for visit_id in visit_ids:
    for det in detectors:
        collection = collection_dict[seqnum_base+visit_id]
        passband = collection_dict_band[seqnum_base+visit_id]
        filedict.write('{:s} {:d} {:d} {:s} {:s}\n'.format('mean_ellipticities_'+formatted_datetime+'.csv', 
                                                 visit_id, det, collection, passband))

filedict.close()

In [7]:
do_make_figures_calexp = False
do_make_ellipticity_in_the_center = True
do_make_mean_ellipticity = True
do_ellipticity_detector_center = True
do_ellipticity_grid = True
regular_grid_or_star_positions = 1 # parametro per l'ellitticità su singolo detector 
# 0: calcolo ellitticità su griglia; 1: calcolo ellitticità su posizioni stelle
n_grid = 3

do_figure_wcs = False
do_figure_bkg = False
do_figure_psf = False
do_fits_preview = False

x_centro_detector = 2000.
y_centro_detector = 2000.

for visitid in visit_ids:

    visitid_complete = visitid+seqnum_base
    # output di tabella di ellitticità per il centro di ogni detector [DVCS]
    visitid_detcenter_output = []
    detector_detcenter_output = []

    xx_rot_detcenter_dvcs_forfigure = []
    yy_rot_detcenter_dvcs_forfigure = []
    ex_detcenter_dvcs_forfigure = []
    ey_detcenter_dvcs_forfigure = []
    e_detcenter_forfigure = []

    xx_rot_grid_dvcs_forfigure = []
    yy_rot_grid_dvcs_forfigure = []
    ex_grid_dvcs_forfigure = []
    ey_grid_dvcs_forfigure = []
    e_grid_forfigure = []
    i_xx_grid_forricardo = []
    i_yy_grid_forricardo = []
    i_xy_grid_forricardo = []
        
    xx_rot_star_dvcs_forfigure = []
    yy_rot_star_dvcs_forfigure = []
    ex_star_dvcs_forfigure = []
    ey_star_dvcs_forfigure = []
    e_star_forfigure = []
    
    # OUTPUT ELLITTICITÀ MEDIE SUI DETECTOR
    detector_detcentermean = []
    visitid_detcentermean = []
    mean_e_detcentermean = []
    median_e_detcentermean = []
    std_e_detcentermean = []
    min_e_detcentermean = []
    max_e_detcentermean = []
    xx_rot_detcentermean_dvcs = []
    yy_rot_detcentermean_dvcs = []
    
    collection = collection_dict[visitid_complete]
    passband = collection_dict_band[visitid_complete]

    # Create the butler
    butler = dafButler.Butler(config, collections=collection)
    datasetType='calexp'

    for detector in detectors:

        print(detector)

        # collection = collection_dict[visitid_complete]
        # passband = collection_dict_band[visitid_complete]

        #Adesso dobbiammo dire al butler che tipo di dati vogliamo.
        #La call si fa chiedendo un datasetType (e.g., deepCoadd, calexp, objectTable) e un data ID(is a dictionary-like identifier for a specific data product)
        #Qui piu' informazioni sul butler 
        #https://github.com/rubin-dp0/tutorial-notebooks/blob/main/04b_Intermediate_Butler_Queries.ipynb

        dataId = {'visit': visitid_complete, 'detector': detector, 'band': passband}
        calexp = butler.get(datasetType, **dataId)
        sources = butler.get('src', dataId)
        psf = calexp.getPsf()
        bkgd = butler.get('calexpBackground', **dataId)
        ccd = calexp.detector.getId()
        det = calexp.getDetector()
        wcs = calexp.getWcs()
        calexp_info = calexp.getInfo()

        rotskypos = (calexp.info.getVisitInfo().getBoresightRotAngle()).asDegrees()
        rottelpos = rsp_to_rtp(rotskypos, \
            (calexp.info.getVisitInfo().getBoresightRaDec())[0].asDegrees(), \
            (calexp.info.getVisitInfo().getBoresightRaDec())[1].asDegrees(), \
            calexp.info.getVisitInfo().getDate().toAstropy()).deg
        rottelpos_radians = np.radians(rottelpos)

        if do_ellipticity_detector_center:

            e_detcenter, ex_detcenter_dvcs, ey_detcenter_dvcs, ex_rot_detcenter_dvcs, ey_rot_detcenter_dvcs, \
                i_xx_detcenter, i_yy_detcenter, i_xy_detcenter, \
                e1, e2, xx_detcenter_dvcs, yy_detcenter_dvcs, theta_detcenter_dvcs, \
                xx_rot_detcenter_dvcs, yy_rot_detcenter_dvcs, ra_detcenter_dvcs, dec_detcenter_dvcs, fwhm, size = \
                calculate_ellipticity_on_xy(calexp, sources, psf, 2, n_grid)
            visitid_detcenter_output = [visitid_complete] * len(e_detcenter)
            detector_detcenter_output = [detector] * len(e_detcenter)
            xx_rot_detcenter_dvcs_forfigure.append(xx_rot_detcenter_dvcs)
            yy_rot_detcenter_dvcs_forfigure.append(yy_rot_detcenter_dvcs)
            ex_detcenter_dvcs_forfigure.append(ex_rot_detcenter_dvcs)
            ey_detcenter_dvcs_forfigure.append(ey_rot_detcenter_dvcs)
            e_detcenter_forfigure.append(e_detcenter)
        
    #################################        
    # Display figures (inizio)
    #################################        
        if do_make_figures_calexp:
            if do_fits_preview:
                fig = plt.figure()
                #Display the image with lsst.afw.display
    
                #The next task is to let AFWDisplay know that we want it to use matplotlib as our default display backend.
                #To do this, we use the setDefaultBackend() function. Remember that we made an alias to lsst.afw.display called afwDisplay in the import
                afwDisplay.setDefaultBackend('matplotlib')
                # get an alias to the lsst.afw.display.Display() method
                display = afwDisplay.Display(frame=fig)
                # set the image stretch algorithm and range
                display.scale('asinh', 'zscale')
                # load the image into the display
                display.mtv(calexp.image)
                # show the corresponding pyplot figure
                plt.title("Image VisitID {:13d} Detector {:03d}".format(visitid_complete,detector))
                plt.show()
                # clean up memory
                remove_figure(fig)

            #Esiste un tutorial per l'utilizzo di  afw_display che il numero 3 del tutorial, da studiare perche' questo e' lo standard, 
            #in particolare fornisce funzioi su come fare i cut, le composizioni di immagini  etc...)
            #https://github.com/rubin-dp0/tutorial-notebooks/blob/main/03a_Image_Display_and_Manipulation.ipynb
            #invece il noteook dopo va su strumenti un po' piu' potenti di data display come firefly che ti apre figure iterattive che credo sia il caso di imparare

            if do_figure_wcs:
                #Figura con WCS
                fig = plt.figure()
                plt.subplot(projection=WCS(calexp.getWcs().getFitsMetadata()))
                calexp_extent = (calexp.getBBox().beginX, calexp.getBBox().endX,
                                 calexp.getBBox().beginY, calexp.getBBox().endY)
                im = plt.imshow(calexp.image.array, cmap='gray', vmin=-200.0, vmax=400,
                                extent=calexp_extent, origin='lower')
                plt.grid(color='white', ls='solid')
                plt.xlabel('Right Ascension')
                plt.ylabel('Declination')
                plt.show()
                remove_figure(fig)

            if do_figure_psf:
                #EXPLORE PSF
                #The PSF object can be used to get a realization of a PSF at a specific point
                fig = plt.figure()
                psfimage = psf.computeImage(PointD(x_centro_detector, y_centro_detector))
                display = afwDisplay.Display()
                display.scale('asinh', min=0.0, max=1.e-3, unit='absolute')
                display.mtv(psfimage)
                plt.title("PSF at 2000 2000 VisitID {:13d} Detector {:03d}".format(visitid_complete,detector))
                plt.show()
                remove_figure(fig)

            #Visualize
            afwDisplay.setDefaultBackend('matplotlib')

            if do_figure_bkg:
                fig = plt.figure()
                afw_display = afwDisplay.Display()
                afw_display.scale('linear', 'zscale')
                afw_display.mtv(bkgd.getImage())
                plt.title("Local Polynomial Background VisitID {:13d} Detector {:03d}".format(visitid_complete,detector))
                plt.show()
                remove_figure(fig)
    #################################        
    #Display figures (FINE)
    #################################        

#     #################################        
#     # Ellipticity on grid su singolo detector (inizio)
#     #################################        
        if True:

            e_star, ex_star_dvcs, ey_star_dvcs, ex_rot_star_dvcs, ey_rot_star_dvcs, i_xx_star, i_yy_star, i_xy_star, \
                e1, e2, xx_star_dvcs, yy_star_dvcs, theta_star_dvcs, \
                xx_rot_star_dvcs, yy_rot_star_dvcs, ra_star_dvcs, dec_star_dvcs, fwhm, size, fluxes_star = \
                calculate_ellipticity_on_xy(calexp, sources, psf, 1, n_grid,
                fileout=folderout+subfolderout_tab+
                'ellipticitymap_'+string_grid_or_star[1]+
                '_visitid{:13d}_det{:03d}_withderotation.csv'.format(visitid_complete, detector))
            visitid_star_output = [visitid_complete] * len(e_star)
            detector_star_output = [detector] * len(e_star)
            xx_rot_star_dvcs_forfigure.append(list(xx_rot_star_dvcs))
            yy_rot_star_dvcs_forfigure.append(list(yy_rot_star_dvcs))
            ex_star_dvcs_forfigure.append(list(ex_rot_star_dvcs))
            ey_star_dvcs_forfigure.append(list(ey_rot_star_dvcs))
            e_star_forfigure.append(list(e_star))

        # Statistica sulle ellitticità
            mean_e = np.mean(e_star)
            median_e = np.median(e_star)
            std_e = np.std(e_star)
            min_e = min(e_star)
            max_e = max(e_star)

        ### plotto la figura con gli ellipticity sticks sulla griglia nel detector            
            fig = plt.figure(figsize=(8, 6))
            plt.quiver(xx_rot_star_dvcs, yy_rot_star_dvcs, 
                       ex_rot_star_dvcs, ey_rot_star_dvcs, e_star, 
                       headlength=0., headwidth=1., pivot='mid', width=0.005)
            colorbar = plt.colorbar(label='e')
            plt.clim(0., max(e_star))
            plt.xlabel('x (DVCS)')
            plt.ylabel('y (DVCS)')
            plt.title('Ellipticity Sticks')
            fig.savefig(folderout+subfolderout_fig+
                        'Ellipticity_Sticks_'+string_grid_or_star[1]+
                        '_'+string_dataset+'_DVCS_visitid{:13d}_det{:03d}_withderotation.{:s}'.format(visitid_complete,detector,format_figures))
            remove_figure(fig)
            
            e_grid, ex_grid_dvcs, ey_grid_dvcs, ex_rot_grid_dvcs, ey_rot_grid_dvcs, i_xx_grid, i_yy_grid, i_xy_grid, \
                e1, e2, xx_grid_dvcs, yy_grid_dvcs, theta_grid_dvcs, \
                xx_rot_grid_dvcs, yy_rot_grid_dvcs, ra_grid_dvcs, dec_grid_dvcs, fwhm, size = \
                calculate_ellipticity_on_xy(calexp, sources, psf, 0, n_grid,
                fileout=folderout+subfolderout_tab+
                'ellipticitymap_'+string_grid_or_star[0]+
                '_visitid{:13d}_det{:03d}_withderotation.csv'.format(visitid_complete, detector))
            
            visitid_grid_output = [visitid_complete] * len(e_grid)
            detector_grid_output = [detector] * len(e_grid)
            xx_rot_grid_dvcs_forfigure.append(xx_rot_grid_dvcs)
            yy_rot_grid_dvcs_forfigure.append(yy_rot_grid_dvcs)
            ex_grid_dvcs_forfigure.append(ex_rot_grid_dvcs)
            ey_grid_dvcs_forfigure.append(ey_rot_grid_dvcs)
            e_grid_forfigure.append(e_grid)
            i_xx_grid_forricardo.append(i_xx_grid)
            i_yy_grid_forricardo.append(i_yy_grid)
            i_xy_grid_forricardo.append(i_xy_grid)

    #################################        
    # Ellipticity on grid su singolo detector (fine)
    #################################

#     #################################
#     #    OUTPUT TABLE MEAN ELLIPTICITIES
#     #################################

        point = Point2D(x_centro_detector, y_centro_detector)
        cam_x, cam_y = pixel_to_camera_angle(point[0], point[1], det)
        xx_rot = np.asarray(cam_x[0])*np.cos(rottelpos_radians) - \
                                np.asarray(cam_y[0])*np.sin(rottelpos_radians)
        yy_rot = np.asarray(cam_x[0])*np.sin(rottelpos_radians) + \
                                np.asarray(cam_y[0])*np.cos(rottelpos_radians)
        
        xx_rot_detcentermean_dvcs.append(yy_rot) #IMPORTANTE INVERTIRE XY TRA CCS E DVCS
        yy_rot_detcentermean_dvcs.append(xx_rot) #IMPORTANTE INVERTIRE XY TRA CCS E DVCS
        detector_detcentermean.append(detector)
        visitid_detcentermean.append(visitid_complete)
        mean_e_detcentermean.append(mean_e)
        median_e_detcentermean.append(median_e)
        std_e_detcentermean.append(std_e)
        min_e_detcentermean.append(min_e)
        max_e_detcentermean.append(max_e)

        del calexp, sources, psf, bkgd, ccd, det, wcs, calexp_info
        del e_detcenter, ex_detcenter_dvcs, ey_detcenter_dvcs, ex_rot_detcenter_dvcs, ey_rot_detcenter_dvcs, \
                i_xx_detcenter, i_yy_detcenter, i_xy_detcenter, \
                e1, e2, xx_detcenter_dvcs, yy_detcenter_dvcs, theta_detcenter_dvcs, \
                xx_rot_detcenter_dvcs, yy_rot_detcenter_dvcs, ra_detcenter_dvcs, dec_detcenter_dvcs, fwhm, size
        del e_grid, ex_grid_dvcs, ey_grid_dvcs, ex_rot_grid_dvcs, ey_rot_grid_dvcs, i_xx_grid, i_yy_grid, i_xy_grid, \
                xx_grid_dvcs, yy_grid_dvcs, theta_grid_dvcs, \
                xx_rot_grid_dvcs, yy_rot_grid_dvcs, ra_grid_dvcs, dec_grid_dvcs
        del e_star, ex_star_dvcs, ey_star_dvcs, ex_rot_star_dvcs, ey_rot_star_dvcs, i_xx_star, i_yy_star, i_xy_star, \
                xx_star_dvcs, yy_star_dvcs, theta_star_dvcs, \
                xx_rot_star_dvcs, yy_rot_star_dvcs, ra_star_dvcs, dec_star_dvcs, fluxes_star
    
    # Figura ellitticità al centro su tutti i detector (una per ogni visitid)
    if do_ellipticity_detector_center:
        fig = plt.figure(figsize=(10,8))
        plt.quiver(xx_rot_detcenter_dvcs_forfigure, yy_rot_detcenter_dvcs_forfigure, 
                   ex_detcenter_dvcs_forfigure, ey_detcenter_dvcs_forfigure, e_detcenter_forfigure,  
                   scale=.5, headlength=0., headwidth=1., pivot='mid', linewidths=.01)

        colorbar = plt.colorbar(label='ellipticity')
        plt.clim(min(e_detcenter_forfigure), max(e_detcenter_forfigure))
        plt.xlim([-figure_size_degrees,figure_size_degrees])
        plt.ylim([-figure_size_degrees,figure_size_degrees])
        plt.xlabel('x [deg]')
        plt.ylabel('y [deg]')
        plt.title('Ellipticity Sticks')
        fig.savefig(folderout+"figures/Ellipticity_Sticks_detcenter_"+string_dataset+
                    "_DVCS_visitid{:13d}_withderotation.{:s}".format(visitid_complete,format_figures))
        remove_figure(fig)

    e_grid_forfigure = np.asarray(e_grid_forfigure).flatten()
    xx_rot_grid_dvcs_forfigure = np.asarray(xx_rot_grid_dvcs_forfigure).flatten()
    yy_rot_grid_dvcs_forfigure = np.asarray(yy_rot_grid_dvcs_forfigure).flatten()
    ex_grid_dvcs_forfigure = np.asarray(ex_grid_dvcs_forfigure).flatten()
    ey_grid_dvcs_forfigure = np.asarray(ey_grid_dvcs_forfigure).flatten()
    i_xx_grid_forricardo = np.asarray(i_xx_grid_forricardo).flatten()
    i_yy_grid_forricardo = np.asarray(i_yy_grid_forricardo).flatten()
    i_xy_grid_forricardo = np.asarray(i_xy_grid_forricardo).flatten()
    
    e_star_forfigure = sum([sublist for sublist in e_star_forfigure], [])
    xx_rot_star_dvcs_forfigure = sum([sublist for sublist in xx_rot_star_dvcs_forfigure], [])
    yy_rot_star_dvcs_forfigure = sum([sublist for sublist in yy_rot_star_dvcs_forfigure], [])
    ex_star_dvcs_forfigure = sum([sublist for sublist in ex_star_dvcs_forfigure], [])
    ey_star_dvcs_forfigure = sum([sublist for sublist in ey_star_dvcs_forfigure], [])
    
    e_star_forfigure = np.asarray(e_star_forfigure).flatten()
    xx_rot_star_dvcs_forfigure = np.asarray(xx_rot_star_dvcs_forfigure).flatten()
    yy_rot_star_dvcs_forfigure = np.asarray(yy_rot_star_dvcs_forfigure).flatten()
    ex_star_dvcs_forfigure = np.asarray(ex_star_dvcs_forfigure).flatten()
    ey_star_dvcs_forfigure = np.asarray(ey_star_dvcs_forfigure).flatten()

    # Tabelle di mappa di ellitticità in formato Ricardo
    df_temp = pd.DataFrame(data={'x': xx_rot_grid_dvcs_forfigure, 
                                 'y': yy_rot_grid_dvcs_forfigure,
                                 'e': e_grid_forfigure,
                                 'theta1': np.degrees(np.arctan2(ey_grid_dvcs_forfigure, ex_grid_dvcs_forfigure)),
                                 'i_xx': i_xx_grid_forricardo,
                                 'i_yy': i_yy_grid_forricardo,
                                 'i_xy': i_xy_grid_forricardo})
    df_temp.to_csv(folderout+subfolderout_tab+
                'ellipticitymap_grid_visitid{:13d}_withderotation_forricardo.csv'.format(visitid_complete), index=None)
    print(folderout+subfolderout_tab+
                'ellipticitymap_grid_visitid{:13d}_withderotation_forricardo.csv'.format(visitid_complete))

    # Tabella di mappa di ellitticità (media su ogni detector)
    xx_rot_detcentermean_dvcs = np.asarray(xx_rot_detcentermean_dvcs).flatten()
    yy_rot_detcentermean_dvcs = np.asarray(yy_rot_detcentermean_dvcs).flatten()
    detector_detcentermean = np.asarray(detector_detcentermean).flatten()
    visitid_detcentermean = np.asarray(visitid_detcentermean).flatten()
    mean_e_detcentermean = np.asarray(mean_e_detcentermean).flatten()
    median_e_detcentermean = np.asarray(median_e_detcentermean).flatten()
    std_e_detcentermean = np.asarray(std_e_detcentermean).flatten()
    min_e_detcentermean = np.asarray(min_e_detcentermean).flatten()
    max_e_detcentermean = np.asarray(max_e_detcentermean).flatten()
    
    df_temp = pd.DataFrame(data={'xx_rot_detcentermean_dvcs': xx_rot_detcentermean_dvcs,
                                'yy_rot_detcentermean_dvcs': yy_rot_detcentermean_dvcs,
                                'detector_detcentermean': detector_detcentermean,
                                'visitid_detcentermean': visitid_detcentermean,
                                'mean_e_detcentermean': mean_e_detcentermean,
                                'median_e_detcentermean': median_e_detcentermean,
                                'std_e_detcentermean': std_e_detcentermean,
                                'min_e_detcentermean': min_e_detcentermean,
                                'max_e_detcentermean': max_e_detcentermean})
    df_temp.to_csv(folderout+subfolderout_tab+
                'ellipticitymap_detcenter_visitid{:13d}_withderotation.csv'.format(visitid_complete), index=None)
    
    # Figura ellitticità su griglia tutti i detector (una per ogni visitid)
    if do_ellipticity_grid:
        plot_ellipticitymap(xx_rot_grid_dvcs_forfigure, yy_rot_grid_dvcs_forfigure, 
                            ex_grid_dvcs_forfigure, ey_grid_dvcs_forfigure, e_grid_forfigure, 
                            folderout+"figures/Ellipticity_Sticks_"+string_grid_or_star[0]+"_"+string_dataset+
                            "_DVCS_visitid{:13d}_withderotation.{:s}".format(visitid_complete,format_figures), 
                            figure_size_degrees=figure_size_degrees, clim_min=0, 
                            clim_max=max(e_grid_forfigure), scale=4.)
        
        plot_ellipticitymap(xx_rot_star_dvcs_forfigure, yy_rot_star_dvcs_forfigure, 
                            ex_star_dvcs_forfigure, ey_star_dvcs_forfigure, e_star_forfigure, 
                            folderout+"figures/Ellipticity_Sticks_"+string_grid_or_star[1]+"_"+string_dataset+
                            "_DVCS_visitid{:13d}_withderotation.{:s}".format(visitid_complete,format_figures), 
                            figure_size_degrees=figure_size_degrees, clim_min=0, 
                            clim_max=max(e_star_forfigure), scale=4.)

# # TABELLA ELLITTICITÀ MEDIE SU FOCALPLANE
# df = pd.DataFrame(data={'detector': detector_detcentermean, 'visitid': visitid_detcentermean, 
#                         'xx_rot_detcentermean_dvcs': xx_rot_detcentermean_dvcs, 
#                         'yy_rot_detcentermean_dvcs': yy_rot_detcentermean_dvcs, 
#                         'mean_e': mean_e_detcentermean, 'median_e': median_e_detcentermean, 
#                         'std_e': std_e_detcentermean, 'min_e': min_e_detcentermean, 
#                         'max_e': max_e_detcentermean})
# df.to_csv(folderout+subfolderout_tab+'mean_ellipticities_DVCS_'+string_dataset+
#           '_'+formatted_datetime+'_withderotation.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
/sdf/data/rubin/shared/image_quality/imsim/ellipticitymap/tables/ellipticitymap_grid_visitid5023071800119_withderotation_forricardo.csv
